### Case Study Groupe 10
Arne Herlinghaus, Max Lütkemeyer, Thomas Mogos, Tim Strauss, Simon Luttmann

# Final Model creation Workflow


#### Imports

In [2]:
import random
import datetime
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from xgboost import XGBClassifier

np.random.seed(42)
random.seed(42)


#### Load original data

In [ ]:
train_csv = "data_6_channels_train.csv" # ← Pfad ggf. anpassen
df = (pd.read_csv(train_csv)
      .rename(columns={"numerical_id": "forest_id",
                       "class": "is_disturbance",
                       "BLU": "blue", "GRN": "green", "RED": "red",
                       "NIR": "near_infrared",
                       "SW1": "shortwave_infrared_1", "SW2": "shortwave_infrared_2"}))

#### Feature engineering

In [ ]:
def engineer_features(group: pd.DataFrame, lags: int = 3) -> pd.DataFrame:
    """Spektrale Indizes + Landsat-8 TCT + 3-Jahres-Stats + Lags/Deltas + Raum/Zeit."""
    eps = 1e-6
    g = group.copy()

    # ---------- Basis-Indizes ----------------------------------------
    g["NDVI"] = (g.near_infrared - g.red) / (g.near_infrared + g.red + eps)
    g["NDMI"] = (g.near_infrared - g.shortwave_infrared_1) / (g.near_infrared + g.shortwave_infrared_1 + eps)
    g["NDWI"] = (g.green - g.near_infrared) / (g.green + g.near_infrared + eps)
    g["NBR"] = (g.near_infrared - g.shortwave_infrared_2) / (g.near_infrared + g.shortwave_infrared_2 + eps)
    g["EVI"] = 2.5 * (g.near_infrared - g.red) / (g.near_infrared + 6 * g.red - 7.5 * g.blue + 1 + eps)
    g["NBR2"] = (g.shortwave_infrared_1 - g.shortwave_infrared_2) / (
                g.shortwave_infrared_1 + g.shortwave_infrared_2 + eps)
    
    # ---------- Trockenheits-Ratio -----------------------------------
    g["SWIR_ratio"] = g.shortwave_infrared_2 / (g.shortwave_infrared_1 + eps)

    # ---------- Landsat-8 Tasseled-Cap (Baig 2014) -------------------
    b2, b3, b4 = g.blue, g.green, g.red
    b5, b6, b7 = g.near_infrared, g.shortwave_infrared_1, g.shortwave_infrared_2
    g["TCB"] = 0.3029 * b2 + 0.2786 * b3 + 0.4733 * b4 + 0.5599 * b5 + 0.5080 * b6 + 0.1872 * b7
    g["TCG"] = -0.2941 * b2 - 0.2430 * b3 - 0.5424 * b4 + 0.7276 * b5 + 0.0713 * b6 - 0.1608 * b7
    g["TCW"] = 0.1511 * b2 + 0.1973 * b3 + 0.3283 * b4 + 0.3407 * b5 - 0.7117 * b6 - 0.4559 * b7
    g["TCT4"] = -0.8239 * b2 + 0.0849 * b3 + 0.4396 * b4 - 0.0580 * b5 + 0.2013 * b6 - 0.2773 * b7
    g["TCT5"] = -0.3294 * b2 + 0.0557 * b3 + 0.1056 * b4 + 0.1855 * b5 - 0.4349 * b6 + 0.8085 * b7
    g["TCT6"] = 0.1079 * b2 - 0.9023 * b3 + 0.4119 * b4 + 0.0575 * b5 - 0.0259 * b6 + 0.0252 * b7

    # --- Interaktions-Features --------------------------------------
    g["TCBxTCG"] = g.TCB * g.TCG
    g["TCBxTCW"] = g.TCB * g.TCW
    g["TCBxNBR"] = g.TCB * g.NBR

    # ---------- 3-Jahres-Median, Std, Anomalie (NDVI & NBR) ----------
    for idx in ("NDVI", "NBR"):
        g[f"{idx}_med3"] = g[idx].rolling(3, min_periods=2).median()
        g[f"{idx}_std3"] = g[idx].rolling(3, min_periods=2).std()
        g[f"{idx}_anom"] = (g[idx] - g[f"{idx}_med3"]) / (g[f"{idx}_med3"].abs() + eps)

    # ---------- Lags & Deltas (blockweise, effizient) ---------------
    base_cols = [
        "NDVI", "NDMI", "NDWI", "NBR", "EVI", "NBR2", "SWIR_ratio",
        "TCB", "TCG", "TCW", "TCT4", "TCT5", "TCT6",
        "TCBxTCG", "TCBxTCW", "TCBxNBR",
        "blue", "green", "red", "near_infrared", "shortwave_infrared_1", "shortwave_infrared_2"
    ]

    lag_features = []
    for lag in range(1, lags + 1):
        shifted = g[base_cols].shift(lag).rename(columns=lambda col: f"l{lag}_{col}")
        deltas = (g[base_cols] - g[base_cols].shift(lag)).rename(columns=lambda col: f"d{lag}_{col}")
        lag_features.extend([shifted, deltas])
    g = pd.concat([g] + lag_features, axis=1)

    # ---------- Raum- & Zeit-Features --------------------------------
    g["year_sin"] = np.sin(2 * np.pi * g.year / 10)
    g["year_cos"] = np.cos(2 * np.pi * g.year / 10)

    return g

In [ ]:
ft_file = "all_data_all_features.csv"

if not Path(ft_file).exists():
    feats = (df.groupby("forest_id")
             .apply(engineer_features)
             .fillna(method="ffill").fillna(method="bfill")
             .reset_index(drop=True))
    feats.to_csv(ft_file, index=False)
else:
    feats = pd.read_csv(ft_file)

feature_cols = [c for c in feats.columns
                if c not in ("forest_id", "year", "is_disturbance")]


#### Test|Validation 80|20 Split with Groups by forest_id

In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
tr_idx, va_idx = next(gss.split(feats, groups=feats["forest_id"]))

train_df, val_df = feats.iloc[tr_idx], feats.iloc[va_idx]
X_train, y_train = train_df[feature_cols].values, train_df["is_disturbance"].values
X_val, y_val = val_df[feature_cols].values, val_df["is_disturbance"].values
print(f"Train {len(train_df):,} | Val {len(val_df):,}")


# ASK TIM

#### Train an XGBoost Ensemble with GroupKFolds based on the train set

In [3]:
#TODO: DESCRIPTION 

In [ ]:

seeds = [42, 1337, 2025, 7, 99]
pos_weights = [1.0, 1.05, 1.1, 1.15]
base_params = dict(objective="binary:logistic",
                   eval_metric="aucpr",
                   eta=0.05,
                   max_depth=7,
                   subsample=0.8,
                   colsample_bytree=0.7,
                   min_child_weight=2,
                   gamma=0.3,
                   alpha=0.7,
                   n_estimators=2000,
                   tree_method="hist",
                   n_jobs=-1,
                   early_stopping_rounds=60)

groups = train_df["forest_id"].values
oof_proba = np.zeros(len(train_df))
for fold, (i_tr, i_va) in enumerate(GroupKFold(5).split(X_train, y_train, groups), 1): # K fold based on forest_id
    X_tr, y_tr = X_train[i_tr], y_train[i_tr]
    X_va, y_va = X_train[i_va], y_train[i_va]
    best, f1_b = np.zeros(len(i_va)), -1
    
    for w in pos_weights:
        proba = np.zeros(len(i_va))

        for s in seeds:
            p = base_params | {"random_state": s,
                               "scale_pos_weight": w * (y_tr == 0).sum() / (y_tr == 1).sum()}# adjust scaling of positive instances for the XGBoost model training based on the class imbalance
            bst = XGBClassifier(**p).fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False)
            iso = CalibratedClassifierCV(bst, cv="prefit", method="isotonic").fit(X_va, y_va)# isotonic regression for better probability estimates
            proba += iso.predict_proba(X_va)[:, 1]
            
        proba /= len(seeds)
        f1_tmp = f1_score(y_va, (proba >= 0.5))
        if f1_tmp > f1_b: best, f1_b = proba, f1_tmp
    oof_proba[i_va] = best
prec, rec, thr = precision_recall_curve(y_train, oof_proba)
tau_star = float(thr[np.argmax(2 * prec[:-1] * rec[:-1] / (prec[:-1] + rec[:-1] + 1e-9))])
print(f"Optimal τ* = {tau_star:.3f}")



We choose the XGBoost Ensemble model because it made the best predictions on our validation set compared to other Models like XGBoost, RandomForest, ExtraTrees, Logistic Regression, KNeighrest Neighboor and SVM. 

#### Retrain xGboost Ensemble Model based on all original forests

In [ ]:
import pickle

full_X = feats[feature_cols].values
full_y = feats["is_disturbance"].values
pos_w  = (full_y == 0).sum() / (full_y == 1).sum()

ensemble = []
for s in seeds:
    params = base_params.copy()
    params.update({
        "random_state"          : s,
        "scale_pos_weight"      : pos_w,
        "early_stopping_rounds" : None
    })

    bst = XGBClassifier(**params).fit(full_X, full_y, verbose=False)

    iso = CalibratedClassifierCV(bst, cv="prefit", method="isotonic")
    iso.fit(full_X, full_y)

    ensemble.append(iso)

artifact = {
    "models"      : ensemble,
    "threshold"   : tau_star,
    "feature_cols": feature_cols,
    "params"      : params,
    "timestamp"   : datetime.datetime.now().isoformat(timespec="seconds"),
}

MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

with open(MODEL_DIR / "xgb_iso_ensemble1.pkl", "wb") as f:
    pickle.dump(artifact, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"✓ Ensemble ({len(ensemble)} Modelle) gespeichert → models/xgb_iso_ensemble.pkl")



══════════════════════════════════════════════════════════════════════════════
TRAIN FULL ENSEMBLE & SAVE
══════════════════════════════════════════════════════════════════════════════


/Users/tim.strauss/PycharmProjects/forest_disturbance_detection/.venv/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/Users/tim.strauss/PycharmProjects/forest_disturbance_detection/.venv/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/Users/tim.strauss/PycharmProjects/forest_disturbance_detection/.venv/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/Users/tim.strauss/PycharmProjects/forest_disturbance_detection/.venv/lib/python3.12/site-p

✓ Ensemble (5 Modelle) gespeichert → models/xgb_iso_ensemble.pkl


In [15]:
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

with open(MODEL_DIR / "xgb_iso_ensemble1.pkl", "wb") as f:
    pickle.dump(artifact, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"✓ Ensemble ({len(ensemble)} Modelle) gespeichert → models/xgb_iso_ensemble.pkl")

✓ Ensemble (5 Modelle) gespeichert → models/xgb_iso_ensemble.pkl


#### Inferenz on test dataset

In [ ]:
TEST_CSV = "data_6_channels_test_pub.csv"
OUT_CSV = "data_6_channels_test_pub_with_predictions.csv"
MODEL_PKL = "models/xgb_iso_ensemble.pkl"

# 6.1  Original-CSV laden (ohne Spaltenänderung merken)
orig_test = pd.read_csv(TEST_CSV)

# 6.2  Für das Modell temporär umbenennen
tmp = (orig_test
       .rename(columns={
    "numerical_id": "forest_id",
    "BLU": "blue", "GRN": "green", "RED": "red",
    "NIR": "near_infrared", "SW1": "shortwave_infrared_1",
    "SW2": "shortwave_infrared_2"})
       .copy())

# Zeilen-/Spalten-Koordinaten für Feature-Engineering
n_cols = 142
tmp["row"] = tmp["forest_id"] // n_cols
tmp["col"] = tmp["forest_id"] % n_cols

# 6.3  Feature-Engineering
test_feat = (tmp.groupby("forest_id")
             .apply(engineer_features)
             .fillna(method="ffill").fillna(method="bfill")
             .reset_index(drop=True))

# 6.4  Ensemble + Threshold laden
with open(MODEL_PKL, "rb") as f:
    art = pickle.load(f)

X_test = test_feat[art["feature_cols"]].values
proba = sum(m.predict_proba(X_test)[:, 1] for m in art["models"]) / len(art["models"])
pred = (proba >= art["threshold"]).astype(int)

# 6.5  Prediction-DataFrame zum Mergen vorbereiten
pred_df = (test_feat[["forest_id", "year"]]
           .assign(is_disturbance=pred)
           .rename(columns={"forest_id": "numerical_id"}))

# 6.6  Mit Original-CSV zusammenführen  (inner-merge garantiert 1-zu-1)
merged = (orig_test
          .merge(pred_df, on=["numerical_id", "year"], how="left"))

# 6.7  Spaltenreihenfolge (wie Bild + neue Spalte)
col_order = ["fid", "year", "numerical_id",
             "BLU", "GRN", "RED", "NIR", "SW1", "SW2",
             "is_disturbance"]
merged = merged[col_order]

# 6.8  Datei schreiben
merged.to_csv(OUT_CSV, index=False)
print(f"✓ '{OUT_CSV}' geschrieben – {merged.shape[0]:,} Zeilen")

# 6.9  Kurzer Überblick
print(f"Label-Verteilung: 0 = {(merged.is_disturbance == 0).sum():,}  |  "
      f"1 = {(merged.is_disturbance == 1).sum():,}")



══════════════════════════════════════════════════════════════════════════════
INFERENCE & MERGE WITH ORIGINAL CSV
══════════════════════════════════════════════════════════════════════════════


/var/folders/9l/s6j5yq4s12j6wvms4084h50h0000gn/T/ipykernel_26023/1338670823.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(engineer_features)
/var/folders/9l/s6j5yq4s12j6wvms4084h50h0000gn/T/ipykernel_26023/1338670823.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")


✓ 'data_6_channels_test_pub_with_predictions.csv' geschrieben – 113,424 Zeilen
Label-Verteilung: 0 = 112,951  |  1 = 473
